# Programming Assignment 2: Movie recommendation

- 과제 목표: 뉴럴 네트워크 모델을 설계한 후 모델을 학습하여 각 영화들의 embedding 들을 생성하고, 영화 embedding 을 활용하여 각 사용자에게 맞춤형 영화를 추천

# Notice

<br>

- 과제를 수행하면서 각 task 마다 꼭 주어진 1개의 cell만을 사용할 필요는 없으며, 여러 개의 cell을 추가하여 자유롭게 사용해도 괜찮습니다.
- 과제 수행을 위해 필요한 module이 있다면 추가로 import 해도 괜찮습니다.

# Import Modules

In [1]:
import warnings, random
import numpy as np
import pandas as pd
import random
import torch # For building network
import torch.nn as nn
import torch.nn.functional as F
from itertools import permutations
from itertools import permutations # For making pairs
warnings.filterwarnings('ignore')

In [2]:
import sys
print(sys.executable)

/Users/vidigummy/opt/anaconda3/envs/RS/bin/python


# Data loading

In [3]:
dir = './MovieLens100K/'
df_ratings = pd.read_csv(dir + 'ratings.csv', usecols=['userId', 'movieId', 'rating'])
df_movies_real = pd.read_csv(dir + 'movies.csv', usecols=['movieId', 'title', 'genres']) # for title-matching

# Preprocessing data

<br>

> ### Problem 1 (3 points)

<br>

1. df_ratings로 부터 각 사용자들이 본 영화를 기록.
2. 사용자 마다 본 영화 목록을 $(movie1, movie2)$, $(movie2, movie1)$ 과 같이 pair로 생성.
    - 즉, 각 사용자 마다 본 영화 목록에 대해 Permutation을 수행
3. 2번 과정이 끝난 후, random을 이용해 각 pair 순서를 무작위로 shuffle.

In [4]:
#### Your Code Here
df_users = df_ratings['userId'].drop_duplicates()
movies_permutation_result = []
df_movies = df_ratings['movieId'] < 1000
df_movies_true = df_ratings[df_movies]
print(df_movies_true)
for i in df_users:
    is_user_movies = df_movies_true['userId'] == i
    user_movies = df_movies_true[is_user_movies]['movieId']
    user_movies_list = user_movies.values.tolist()
    user_movies_permutation = list(permutations(user_movies_list, 2))
    movies_permutation_result.extend(user_movies_permutation)
random.shuffle(movies_permutation_result)
# print(len(movies_permutation_result))
# print(movies_permutation_result)
# print(len(movies_permutation_result))

       userId  movieId  rating
0           1        1     4.0
1           1        3     4.0
2           1        6     4.0
3           1       47     5.0
4           1       50     5.0
...       ...      ...     ...
99590     610      912     3.5
99591     610      919     3.5
99592     610      923     4.0
99593     610      924     4.5
99594     610      968     4.0

[21667 rows x 3 columns]


# Build and train neural networks for generating movie embeddings

<br>

> ### Problem 2 (4 points)

<br>

- 각 영화 임베딩을 구하기 위해 뉴럴 네트워크 모델을 활용하여 multi-class classification 을 수행

- 설계할 신경망의 기본 구조는 **Input Layer - Hidden(Embedding) Layer - Output Layer**.
    - 7주차 강의자료 p.7 신경망 구조 이미지 참고

- 현재 Network를 통해 하고자 하는 task는 multi-class classification.
    - 예: $(movie1, movie2)$ 와 같은 입력 데이터와 정답 출력 데이터를 이용해 모델을 학습
        - Input : $movie1$의 one-hot vector
        - Output : $\widehat{movie2}$의 one-hot vector
        - Compute Loss : $\widehat{movie2}$ 와 $movie2$ 간의 Cross-entropy Loss
- 학습이 완료된 이후에 input layer와 hidden(embedding) layer 사이의 weight matrix $W_{in}$를 movie에 대한 embedding vector로 사용이 가능.
> embedding size(# of hidden units)는 100 이하로 두는 것을 권장. <br>
> embedding layer 다음 hidden layer를 더 추가하여 Genre와 같은 추가 정보를 학습에 활용 할 수도 있음 (필수적으로 고려해야할 사항은 아님).

- 설계한 뉴럴 네트워크 모델의 학습이 완료된 후, 학습된 weight matrix $W_{in}$의 행/열벡터를 각 영화에 대한 embedding vector로 간주하여 영화 embedding 들을 구할 수 있음.

In [5]:
class NnNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.embedded_layer = nn.Linear(input_size, 100)
        self.hidden_layer = nn.Linear(100, 100)
        self.output_layer = nn.Linear(100,input_size)
        self.relu = nn.ReLU()
        self.activation = nn.Sigmoid()
    def forward(self,x):
        x = self.embedded_layer(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.activation(x)
        return x

In [6]:
#### Your Code Here
movies_pair_dataframe = pd.DataFrame(data= movies_permutation_result, columns=['movie1','movie2'])

#영화의 개수를 가지고 one-hot vector로 만들어뿌자
df_movie1 = pd.DataFrame(data=movies_pair_dataframe['movie1'].values,columns=['movieId'])
print(df_movie1)
df_movie2 = pd.DataFrame(data=movies_pair_dataframe['movie2'].values,columns=['movieId'])
df_concat_movies = pd.concat([df_movie1,df_movie2],axis=0)
df_one_hot_embedded = pd.get_dummies(data=df_concat_movies, columns = ['movieId'])
df_one_hot_embedded_movie1 = df_one_hot_embedded.iloc[:int(df_one_hot_embedded.shape[0]/2), :]
df_one_hot_embedded_movie2 = df_one_hot_embedded.iloc[int(df_one_hot_embedded.shape[0]/2):,:]
#df로 만들었다. (1099200, 1859)로 자식들이 아주 건강하다.
torch_one_hot_movie1 = torch.as_tensor(df_one_hot_embedded_movie1.values).float()
torch_one_hot_movie2 = torch.as_tensor(df_one_hot_embedded_movie2.values).float()
print("torch_one_hot_movie1 shape : ", torch_one_hot_movie1.shape)
model = NnNetwork(torch_one_hot_movie1.shape[1])
loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# optimizer2 = torch.optim.Adam(model.parameters(),lr=0.1, betas = (0.9,0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer2 = torch.optim.Adam(model.parameters(),lr=0.1, betas = (0.9,0.999), eps=1e-08, weight_decay=0, amsgrad=False)

for epoch in range(2):
    #make one-hot vector(input, target Generation)
    output = model(torch_one_hot_movie1)
    loss = loss_function(output, torch_one_hot_movie2)
    optimizer2.zero_grad()
    loss.backward()
    optimizer2.step()
    # loss.backward()
    print('Epoch {:4d}/{} loss: {:.6f}'.format(
        epoch+1, 2, loss.item()
    ))
    



         movieId
0            155
1             72
2            553
3            508
4            775
...          ...
1869541      920
1869542      741
1869543      587
1869544      329
1869545      973

[1869546 rows x 1 columns]
torch_one_hot_movie1 shape :  torch.Size([1869546, 761])
Epoch    1/2 loss: 6.635469
Epoch    2/2 loss: 6.366889


In [7]:
movie_embedding = dict()
cnt = 0
# print(df_movie1)
#761*100
for param in model.named_parameters():
    a = param[1]
    print(len(a[0]))
    print(len(a))
    for i in range(len(a[0])):
        # print(i)
        # if(int(df_movie1.loc[i,'movieId']) == 70):
        #     print(70)
        print(int(df_movie1.loc[i,'movieId']))
        movie_embedding[int(df_movie1.loc[i,'movieId'])] = a[:,i]
    break
print(len(movie_embedding))
# print(movie_embedding)


761
100
155
72
553
508
775
891
594
550
780
113
34
541
10
15
610
3
940
380
262
589
780
19
595
509
420
589
368
880
318
750
778
597
543
349
599
135
339
477
333
948
784
745
736
252
242
60
374
374
553
151
533
836
362
616
234
357
938
249
786
150
583
778
235
707
842
318
164
95
247
546
370
252
16
235
97
466
897
606
590
555
918
849
522
420
575
296
425
193
266
782
316
168
858
277
474
266
344
747
412
480
714
922
10
292
480
3
357
207
36
858
213
858
661
500
555
95
457
928
24
784
158
70
733
524
186
597
105
207
500
923
198
500
435
364
370
39
457
920
708
780
348
589
247
588
1
70
356
969
500
745
588
892
158
839
230
327
53
293
867
500
778
661
733
70
454
543
261
988
318
227
829
553
314
17
188
588
520
539
541
780
628
382
587
29
810
50
520
65
410
996
185
256
370
432
155
490
564
163
455
574
595
953
169
916
858
597
50
364
151
416
349
555
708
277
60
356
153
688
193
318
508
292
908
110
358
910
481
431
25
273
39
208
254
344
380
12
172
577
293
28
32
555
837
256
32
597
780
631
663
62
261
924
95
382
377
205
628
58

# Recommend customized movies to user

<br>

> ### Problem 3 (3 points)

<br>

- 임의의 한명의 사용자에 대하여 해당 사용자가 봤던 영화 n개에 대해 **통합된 embedding vector**를 생성.
    - n개의 embedding vector들에 대해, element-wise한 계산을 통해 통합된 하나의 embedding vector를 생성.
    - 이 embedding vector는 해당 사용자의 전반적인 영화 시청 성향을 나타내는 embedding vector로 간주할 수 있음.
    - 즉, **사용자 1명 당 1개의 embedding vector**를 가짐.
- 통합된 embedding vector와 학습된 weight matrix $W_{in}$의 모든 영화 embedding vector들 간의 유사도를 계산.
- 그 중 유사도가 높은 (top n) 영화들을 선정, 사용자에게 추천.
    > Recommended format : MovieId, Title, Genre, Similarity 가 포함된 형식

In [8]:
#### Your Code Here
is_user_movies = df_movies_true['userId'] == 1
users_movies = df_movies_true[is_user_movies]
user_movies = users_movies['movieId']
# print(user_movies.shape)
user_movie_dict =dict()
# print(df_movie1[df_movie1['movieId'] == 157])
# for movie in movie_embedding:
#     print(movie)
for movieId in user_movies:
    # print(movie_embedding[movieId])
    try:
        user_movie_dict[movieId] = movie_embedding[movieId]
    except:
        print(movieId , " has lost!")
# print(user_movie_dict)
print(len(user_movie_dict))
user_embedding_vector = torch.zeros(100)
print(user_embedding_vector)
for i in user_movie_dict.keys():
    # print(len(i_list))
    i_list = user_movie_dict[i]/len(user_movie_dict)
    for j in range(0,len(i_list)):
        user_embedding_vector[j] += i_list[j]

#유저 임베딩 벡터 완성(이게 맞나...)
print(user_embedding_vector)



101  has lost!
157  has lost!
223  has lost!
423  has lost!
441  has lost!
673  has lost!
943  has lost!
954  has lost!
41
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([-0.0206,  0.0352, -0.1359,  0.0833,  0.1099, -0.0077,  0.0744, -0.1205,
        -0.0140,  0.0969, -0.0546, -0.0667, -0.0571,  0.0057,  0.0875,  0.0742,
        -0.0125, -0.0564,  0.0116, -0.0933, -0.0158, -0.0058, -0.0882, -0.0444,
         0.0015, -0.0168, -0.0282, -0.0143, -0.0361, -0.0097,  0.1060,  0.0812,
         0.1279, -0.0011, -0.0555,  0.0878,  0.0882,  0.0146, -0.0280, -0.0411,
        -0.0067,  0.0694, -0.0320,  

In [9]:
# print(movie_embedding)
result =dict()
for embedded_movie in movie_embedding.keys():
    cosine_sim = F.cosine_similarity(movie_embedding[embedded_movie], user_embedding_vector, dim = 0)
    # print(embedded_movie, " : ",cosine_sim)
    result[embedded_movie] = cosine_sim

result_recommend_list = sorted(result.items(), reverse=True, key=lambda item: item[1])
# print(result_recommend_dict)
# for i in result_recommend_dict:
#     print(i[0])
result_list_top_ten = list()
result_list_top_ten_sim = list()
result_recommend_list_top_ten = result_recommend_list[0:10]
for i in result_recommend_list_top_ten:
    result_list_top_ten.append(i[0])
    result_list_top_ten_sim.append(i[1])
# print(result_list_top_ten)
# print(df_movies_real[df_movies_real['movieId'].isin(result_list_top_ten)])
for ten in range (0,len(result_list_top_ten)):
    one_thing_title = df_movies_real[df_movies_real['movieId'] == result_list_top_ten[ten]]['title'].values[0]
    one_thing_genres = df_movies_real[df_movies_real['movieId'] == result_list_top_ten[ten]]['genres'].values
    # print(ten," ", one_thing_title, " ", one_thing_genres)
    print("movieId : ",result_list_top_ten[ten], " | title : ", one_thing_title, "| genres : ",one_thing_genres,"| sim :", result_list_top_ten_sim[ten].item())



movieId :  736  | title :  Twister (1996) | genres :  ['Action|Adventure|Romance|Thriller'] | sim : 0.8748392462730408
movieId :  905  | title :  It Happened One Night (1934) | genres :  ['Comedy|Romance'] | sim : 0.8631466031074524
movieId :  918  | title :  Meet Me in St. Louis (1944) | genres :  ['Musical'] | sim : 0.8585366010665894
movieId :  968  | title :  Night of the Living Dead (1968) | genres :  ['Horror|Sci-Fi|Thriller'] | sim : 0.8440254926681519
movieId :  70  | title :  From Dusk Till Dawn (1996) | genres :  ['Action|Comedy|Horror|Thriller'] | sim : 0.8412495255470276
movieId :  422  | title :  Blink (1994) | genres :  ['Thriller'] | sim : 0.8407755494117737
movieId :  135  | title :  Down Periscope (1996) | genres :  ['Comedy'] | sim : 0.8376667499542236
movieId :  218  | title :  Boys on the Side (1995) | genres :  ['Comedy|Drama'] | sim : 0.8374474048614502
movieId :  491  | title :  Man Without a Face, The (1993) | genres :  ['Drama'] | sim : 0.8355104327201843
movie

잘 모르겠습니다. 죄송합니다. 부끄러운 말씀이지만, 무엇을 해야할 지에 대해서는 이해했습니다. 하지만 머신러닝을 위한 python 코드는 처음 짜는 것이기도 하고, 어떤 것이 좋은지도 전혀 알지 못하는 상황에서 이 과제를 해결하기에는 능력이 부족하다는 판단입니다. 
분명 오랜 시간을 투자했고, 여러모로 찾아봤지만 제 능력과 시간 내에서는 해결을 할 수 없다는 결론입니다.
다음 과제가 나올지는 잘 모르겠으나, 다음에는 중간 과정마다 어떠한 결과가 나오는 것이 좋은 것일지 알려주실 수 있을까요? 이또한 무례한 것이라면 사과 드리겠으나, 머신러닝 코드 에 대한 이해가 부족한 저로서는 이것이 최소한의 요청일듯합니다. 죄송합니다.